In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,14)

In [2]:
import hashlib
import re

# patterns for matching 3 and 5 chars in a row
pattern_chars ={
    3: re.compile(r'(.)\1\1'),
    5: re.compile(r'(.)\1\1\1\1')
}
check_sums = {
}
def checksum(number,seed):
    check_string = f"{seed}{number}"
    if check_string in check_sums:
        return check_sums[check_string]
    # Get the hexadecimal representation of the digest
    return hashlib.md5(check_string.encode('utf-8')).hexdigest()

def expanded_checksum(number,seed):
    tmp_checksum = checksum(number,seed)
    for _ in range(2016):
        if tmp_checksum in check_sums:
            tmp_checksum = check_sums[tmp_checksum]
        else:
            tmp_checksum = hashlib.md5(tmp_checksum.encode('utf-8')).hexdigest()
    return tmp_checksum


def find_chars(input_string, count):
    # Define the regex pattern for exactly count consecutive identical characters
    if count not in pattern_chars:
        return None
    pattern = pattern_chars[count]
    matched_chars =[]
    for match in pattern.finditer(input_string):
        isolated = True
        #Test character before (this shouldn't really happen, but checking anyway)
        span = match.span()
        # short circuit 5's
        if count == 3:
            if span[0] > 0:
                if input_string[span[0]-1] == match.group(1):
                    isolated = False
            # check character after
            if span[1] < len(input_string):
                if input_string[span[0]+count] == match.group(1):
                    isolated = False
            #if isolated:
            matched_chars.append(match.group(1))
            break
        else:
            matched_chars.append(match.group(1))
    return matched_chars


In [7]:
from collections import deque
def find_key(seed,protocol):
    # my_keys: list of tuple(idx,char,checksum)
    my_keys = set()
    
    # potentials: set of tuple(idx,char,checksum)
    potentials = deque(maxlen=1001)
    
    #initialize idx
    idx = 0
    # okay, just a bit frustrated here, I was stopping at 64, and you need to go a bit beyond then sort the results
    while len(my_keys) < 72:
        # get md5sum
        if protocol == 1:
            my_checksum = checksum(idx,seed)
        else:
            my_checksum = expanded_checksum(idx,seed)
        # get all matching 5 char set
        my_chars = find_chars(my_checksum,5)
        # did we match?
        if my_chars:
            # walk potentials to see if there is a match
            for potential in potentials:
                # is it stale?
                if potential[0] < idx <= potential[0]+ 1001 and potential[1] in my_chars:
                    # yes, add it to my_keys
                    #my_keys.add((potential,(idx,tuple(my_chars),my_checksum)))
                    my_keys.add(potential)
        # get first matching 3 char set
        my_chars = find_chars(my_checksum,3)
        # did we match?
        if my_chars:
            # yes, add it to potentials
            potentials.append((idx,my_chars[0],my_checksum))
        idx+=1
    return sorted(list(my_keys))[63][0]


# set seed
seed = {
    1: 'ngcjuoqr',
    2: 'ngcjuoqr'
}
import time

for part in [1, 2]:
    start = time.time()
    print(f"Part {part}:  {find_key(seed[part],part)}, took {time.time() - start} seconds")


    

Part 1:  18626, took 0.2671530246734619 seconds
Part 2:  20092, took 190.40072655677795 seconds


In [9]:
checksum(0,'abc')

'577571be4de9dcce85a041ba0410f29f'

In [11]:
expanded_checksum(0,'abc')

'a107ff634856bb300138cac6568c0f24'